<a href="https://colab.research.google.com/github/rajatyadav1998/semantic-search-engine/blob/main/semantic_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install -U sentence-transformers


  Using cached sentence_transformers-5.0.0-py3-none-any.whl.metadata (16 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
Using cached sentence_transformers-5.0.0-py3-none-any.whl (470 kB)
Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl (36

In [7]:
import pandas as pd

# Load the uploaded file (adjust filename if needed)
df_questions = pd.read_csv("Questions.csv", usecols=["Id", "Title", "Body", "CreationDate"], nrows=50000)

# Display first few rows
df_questions.head()


,Id,CreationDate,Title,Body
0,80,2008-08-01T13:57:07Z,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,2008-08-01T14:41:24Z,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,2008-08-01T15:50:08Z,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2008-08-01T18:42:19Z,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,2008-08-01T23:22:08Z,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [9]:
from sentence_transformers import SentenceTransformer

# Load a small, fast, effective sentence embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
# Extract question titles
titles = df_questions['Title'].astype(str).tolist()

# Generate dense vector embeddings
embeddings = model.encode(titles, show_progress_bar=True)


Batches:   0%|          | 0/1563 [00:00<?, ?it/s]

In [11]:
import numpy as np

np.save("question_titles.npy", titles)
np.save("question_embeddings.npy", embeddings)


In [12]:
!pip install fastapi nest-asyncio pyngrok uvicorn


In [13]:
from fastapi import FastAPI
from pydantic import BaseModel
import numpy as np
import uvicorn
import nest_asyncio
from pyngrok import ngrok


In [14]:
app = FastAPI()

titles = np.load("question_titles.npy", allow_pickle=True)
embeddings = np.load("question_embeddings.npy", allow_pickle=True)

class QueryRequest(BaseModel):
    query: str

@app.post("/search")
def search_questions(req: QueryRequest):
    from sklearn.metrics.pairwise import cosine_similarity
    from sentence_transformers import SentenceTransformer

    model = SentenceTransformer("all-MiniLM-L6-v2")
    query_embedding = model.encode([req.query])

    scores = cosine_similarity(query_embedding, embeddings)[0]
    top_indices = scores.argsort()[::-1][:5]
    top_titles = titles[top_indices]

    return {"top_results": top_titles.tolist()}


In [15]:
# Enable nested event loop
nest_asyncio.apply()

# Expose the FastAPI app to the internet using ngrok
public_url = ngrok.connect(8000)
print(f"Your app is live at: {public_url}")

# Start the server
uvicorn.run(app, port=8000)


ERROR:pyngrok.process.ngrok:t=2025-07-25T06:55:45+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-07-25T06:55:45+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-07-25T06:55:45+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [16]:
!ngrok config add-authtoken 30M4sFq07MnnJN7ao11As3FeOoo_64mq5pD6GSnPzk62CY2N


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [17]:
public_url = ngrok.connect(9200)
print(f"Your app is live at: {public_url}")


Your app is live at: NgrokTunnel: "https://823cb30c9385.ngrok-free.app" -> "http://localhost:9200"


In [18]:
from fastapi import FastAPI
from pydantic import BaseModel
import numpy as np

app = FastAPI()

titles = np.load("question_titles.npy", allow_pickle=True)
embeddings = np.load("question_embeddings.npy", allow_pickle=True)

from sklearn.metrics.pairwise import cosine_similarity

class Query(BaseModel):
    question: str

from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2")

@app.post("/predict")
def predict(query: Query):
    query_embedding = model.encode([query.question])
    similarity_scores = cosine_similarity(query_embedding, embeddings)[0]
    top_index = similarity_scores.argmax()
    return {
        "most_similar_question": titles[top_index],
        "similarity_score": float(similarity_scores[top_index])
    }


In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

nest_asyncio.apply()
public_url = ngrok.connect(9200)
print("Public URL:", public_url)

uvicorn.run(app, port=9200)


Public URL: NgrokTunnel: "https://487ac8fd3afd.ngrok-free.app" -> "http://localhost:9200"


INFO:     Started server process [181]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:9200 (Press CTRL+C to quit)


INFO:     103.46.203.237:0 - "GET /docs HTTP/1.1" 200 OK


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-7' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:69> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 67, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    se

INFO:     103.46.203.237:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     103.46.203.237:0 - "POST /search HTTP/1.1" 200 OK
INFO:     103.46.203.237:0 - "POST /search HTTP/1.1" 200 OK
INFO:     103.46.203.237:0 - "POST /search HTTP/1.1" 200 OK


In [ ]:
import requests

url = "https://487ac8fd3afd.ngrok-free.app/search"
data = {"query": "how to use pandas dataframe"}

headers = {
    'Content-Type': 'application/json',
    'ngrok-skip-browser-warning': 'true'
}

try:
    response = requests.post(url, json=data, headers=headers, timeout=10)
    print("Status Code:", response.status_code)

    if response.status_code == 200:
        print("Response JSON:")
        print(response.json())
    else:
        print("Something went wrong:", response.text)

except requests.exceptions.RequestException as e:
    print("Request failed:", e)

